In [0]:
from pyspark.sql import Row, Column
from pyspark.sql.functions import *
from pyspark.sql.types import *

##Prikupljanje i transformacija podataka

Prikupljeni su podaci o svetskom prvenstvu u fudbalu, odrzanog ove jeseni u Kataru.Razlikujemo podatke o samim utakmicama, o pojedinacnim dostignucima svakog igraca kao i o posecenosti svih utakmica na prvenstvu.U daljem kodu, podaci su transformisani i na taj nacin prilagodjeni za dalju analizu.

In [0]:
file_location = "/FileStore/tables/FIFA_WC_2022_Player_stats.csv"

df = spark.read.option("inferSchema", "true").option("header", "true").csv(file_location, sep=',')
df.show(5, truncate=False)

+--------------------+--------+---------+------+----------+----------+-----+------------+-------+-----------+-----+-------+----------+---------+--------+------------+---------+-----------+-------------+-------------------+----------------+------------------------+---+----+---------+--------------+--------+---------------+------------------+----------+--------------------+
|player              |position|team     |age   |club      |birth_year|games|games_starts|minutes|minutes_90s|goals|assists|goals_pens|pens_made|pens_att|cards_yellow|cards_red|goals_per90|assists_per90|goals_assists_per90|goals_pens_per90|goals_assists_pens_per90|xg |npxg|xg_assist|npxg_xg_assist|xg_per90|xg_assist_per90|xg_xg_assist_per90|npxg_per90|npxg_xg_assist_per90|
+--------------------+--------+---------+------+----------+----------+-----+------------+-------+-----------+-----+-------+----------+---------+--------+------------+---------+-----------+-------------+-------------------+----------------+-----------

Izbacujemo kolone koje nam nece biti znacajne u analizi.

In [0]:
df = df.drop("npxg_per90", "npxg_xg_assist_per90","xg_xg_assist_per90", "xg_assist_per90", "xg_per90", "npxg_xg_assist","xg_assist", "npxg", "goals_assists_pens_per90", "goals_pens_per90", "goals_assists_per90", "games_starts", "minutes_90s", "goals_pens", "pens_made", "xg", "assists_per90", "pens_att")
df.show(5, truncate=False)

+--------------------+--------+---------+------+----------+----------+-----+-------+-----+-------+------------+---------+-----------+
|player              |position|team     |age   |club      |birth_year|games|minutes|goals|assists|cards_yellow|cards_red|goals_per90|
+--------------------+--------+---------+------+----------+----------+-----+-------+-----+-------+------------+---------+-----------+
|Aaron Mooy          |MF      |Australia|32-094|Celtic    |1990      |4    |360    |0    |0      |1           |0        |0.0        |
|Aaron Ramsey        |MF      |Wales    |31-357|Nice      |1990      |3    |266    |0    |0      |1           |0        |0.0        |
|Abdelhamid Sabiri   |MF      |Morocco  |26-020|Sampdoria |1996      |5    |181    |0    |1      |1           |0        |0.0        |
|Abdelkarim Hassan   |DF      |Qatar    |29-112|Al Sadd SC|1993      |3    |270    |0    |0      |0           |0        |0.0        |
|Abderrazak Hamdallah|FW      |Morocco  |32-001|Al-Ittihad|199

Izbacujemo dane iz kolone 'age' i ostavljamo samo godine. Takodje menjamo imena kolona 'team' i 'goals_per90' kako bi imali bolji uvid u ono sto pokazuju.

In [0]:

df = df.withColumn('age', split(df['age'], '-').getItem(0)) 
df.show(5, truncate=False)

+--------------------+--------+---------+---+----------+----------+-----+-------+-----+-------+------------+---------+-----------+
|player              |position|team     |age|club      |birth_year|games|minutes|goals|assists|cards_yellow|cards_red|goals_per90|
+--------------------+--------+---------+---+----------+----------+-----+-------+-----+-------+------------+---------+-----------+
|Aaron Mooy          |MF      |Australia|32 |Celtic    |1990      |4    |360    |0    |0      |1           |0        |0.0        |
|Aaron Ramsey        |MF      |Wales    |31 |Nice      |1990      |3    |266    |0    |0      |1           |0        |0.0        |
|Abdelhamid Sabiri   |MF      |Morocco  |26 |Sampdoria |1996      |5    |181    |0    |1      |1           |0        |0.0        |
|Abdelkarim Hassan   |DF      |Qatar    |29 |Al Sadd SC|1993      |3    |270    |0    |0      |0           |0        |0.0        |
|Abderrazak Hamdallah|FW      |Morocco  |32 |Al-Ittihad|1990      |4    |68     |0 

In [0]:
df = df.withColumnRenamed("team", "national team")\
       .withColumnRenamed("goals_per90", "goals_per_game")
df.show(10, truncate=False)

+---------------------+--------+-------------+---+------------+----------+-----+-------+-----+-------+------------+---------+--------------+
|player               |position|national team|age|club        |birth_year|games|minutes|goals|assists|cards_yellow|cards_red|goals_per_game|
+---------------------+--------+-------------+---+------------+----------+-----+-------+-----+-------+------------+---------+--------------+
|Aaron Mooy           |MF      |Australia    |32 |Celtic      |1990      |4    |360    |0    |0      |1           |0        |0.0           |
|Aaron Ramsey         |MF      |Wales        |31 |Nice        |1990      |3    |266    |0    |0      |1           |0        |0.0           |
|Abdelhamid Sabiri    |MF      |Morocco      |26 |Sampdoria   |1996      |5    |181    |0    |1      |1           |0        |0.0           |
|Abdelkarim Hassan    |DF      |Qatar        |29 |Al Sadd SC  |1993      |3    |270    |0    |0      |0           |0        |0.0           |
|Abderrazak H

In [0]:
df.printSchema()

root
 |-- player: string (nullable = true)
 |-- position: string (nullable = true)
 |-- national team: string (nullable = true)
 |-- age: string (nullable = true)
 |-- club: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- games: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- cards_yellow: integer (nullable = true)
 |-- cards_red: integer (nullable = true)
 |-- goals_per_game: double (nullable = true)



Upisujemo podatke

In [0]:
path = 'dbfs:/projekat/'
csv_name = path + 'dfGames2.csv'
parquet_name = path + 'dfGames2.parquet'
orc_name = path + 'dfGames2.orc'


print(path,csv_name,parquet_name,orc_name,sep='\n')
     

df2.write.csv(csv_name, header=True)
df2_csv = spark.read.csv(csv_name, header=True, inferSchema=True)
df2_csv.printSchema()

df2.write.save(parquet_name, format='parquet') 
spark.read.load(parquet_name, format='parquet')

dbfs:/projekat/
dbfs:/projekat/dfGames2.csv
dbfs:/projekat/dfGames2.parquet
dbfs:/projekat/dfGames2.orc
root
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- final result: string (nullable = true)
 |-- total goals: integer (nullable = true)
 |-- possession team1: string (nullable = true)
 |-- possession team2: string (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- category: string (nullable = true)

Out[329]: DataFrame[team1: string, team2: string, final result: string, total goals: int, possession team1: string, possession team2: string, date: date, time: string, category: string]

Ucitavamo podatke o utakmicama i nakon toga izbacujemo veci deo kolona, jer nam nece biti potrebne za analizu.

In [0]:
#/FileStore/tables/FIFA_WC_2022_Games-2.csv

file_location = "/FileStore/tables/FIFA_WC_2022_Games-2.csv"

df2 = spark.read.option("inferSchema", "true").option("header", "true").csv(file_location, sep=',')
df2.show(5, truncate=False)

+-------------+------------+----------------+----------------+---------------------+---------------------+---------------------+-----------+-------+--------+--------------------+--------------------+--------------+--------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+-------------+-------------+------------------------+------------------------+-------------------------+-------------------------+--------------------------------------+---------------------------------------+----------------------------------------+----------------------------------------+------------------+------------------+-------------------------+-------------------------+---------------------+---------------------+--------------------------+--------------------------+-------------------+-------------------+-----------------------------+-----------------------------+--------------------------------+-----------------

In [0]:
df2 = df2.drop("possession in contest", "total attempts team1","total attempts team2", "goal inside the penalty area team1", "goal inside the penalty area team2", "goal outside the penalty area team1","goal outside the penalty area team2", "assists team1", "assists team2", "on target attempts team1", "on target attempts team2", "off target attempts team2", "off target attempts team1", "attempts inside the penalty area team1", "attempts inside the penalty area team2", "attempts outside the penalty area  team1", "attempts outside the penalty area  team2", "attempts inside the penalty area  team2", "left channel team1", "left channel team2", "left inside channel team1", "left inside channel team2")
df2.show(5, truncate=False)

+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+--------------+--------------+---------------------+---------------------+--------------------------+--------------------------+-------------------+-------------------+-----------------------------+-----------------------------+--------------------------------+--------------------------------+---------------------------------+---------------------------------+-------------------------------+-------------------------------+-----------------------------------------------------+-----------------------------------------------------+---------------------------+---------------------------+--------------------------+---------------------------+-------------------------------------+-------------------------------------+------------------------------------+-------------------------------------+------------------+------------------+---------------+---------------

In [0]:
df2 = df2.drop("central channel team1", "central channel team2","right inside channel team1", "right inside channel team2", "right channel team1", "right channel team2","total offers to receive team1", "total offers to receive team2", "inbehind offers to receive team1", "inbehind offers to receive team2", "inbetween offers to receive team1", "inbetween offers to receive team2", "infront offers to receive team1", "infront offers to receive team2", "receptions between midfield and defensive lines team1", "receptions between midfield and defensive lines team2", "attempted line breaks team1", "attempted line breaks team2", "completed line breaksteam1", "completed line breaks team2", "attempted defensive line breaks team1", "attempted defensive line breaks team2")
df2.show(5, truncate=False)

+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+--------------+--------------+------------------------------------+-------------------------------------+------------------+------------------+---------------+---------------+-------------------+-------------------+--------------+--------------+------------+------------+----------------------+----------------------+-------------+-------------+-----------------------+-----------------------+--------------------------------+--------------------------------+-------------+-------------+----------------+----------------+----------------------+----------------------+----------------------+----------------------+---------------+---------------+----------------------+----------------------+---------------------------------+---------------------------------+
|team1        |team2       |possession team1|possession team2|number of goals team1|number of goals team2

In [0]:
df2 = df2.drop("completed defensive line breaksteam1", "completed defensive line breaks team2","yellow cards team1", "yellow cards team2", "red cards team1", "red cards team2","fouls against team1", "fouls against team2", "offsides team1", "offsides team2", "passes team1", "passes team2", "passes completed team1", "passes completed team2", "crosses team1", "crosses team2", "crosses completed team1", "crosses completed team2", "switches of play completed team1", "switches of play completed team2", "corners team1", "corners team2", "free kicks team1", "free kicks team2", "penalties scored team1", "penalties scored team2", "goal preventions team1", "goal preventions team2", "own goals team1", "own goals team2", "forced turnovers team1", "forced turnovers team2", "defensive pressures applied team1", "defensive pressures applied team2", "conceded team1", "conceded team2")
df2.show(5, truncate=False)

+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+
|team1        |team2       |possession team1|possession team2|number of goals team1|number of goals team2|date       |hour   |category|
+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+
|QATAR        |ECUADOR     |42%             |50%             |0                    |2                    |20 NOV 2022|17 : 00|Group A |
|ENGLAND      |IRAN        |72%             |19%             |6                    |2                    |21 NOV 2022|14 : 00|Group B |
|SENEGAL      |NETHERLANDS |44%             |45%             |0                    |2                    |21 NOV 2022|17 : 00|Group A |
|UNITED STATES|WALES       |51%             |39%             |1                    |1                    |21 NOV 2022|20 : 00|Group B |
|ARGENTINA    |SAUDI ARABIA|64%             |24%

In [0]:
df2.printSchema()

root
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- possession team1: string (nullable = true)
 |-- possession team2: string (nullable = true)
 |-- number of goals team1: integer (nullable = true)
 |-- number of goals team2: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- category: string (nullable = true)



Menjamo imena nekih kolona, a nakon toga u kolonama 'team1' i 'team2' smanjujemo slova i ostavljamo samo pocetno slovo veliko.

In [0]:
df2 = df2.withColumnRenamed("hour", "time")
df2.show(5, truncate=False)

+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+
|team1        |team2       |possession team1|possession team2|number of goals team1|number of goals team2|date       |time   |category|
+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+
|QATAR        |ECUADOR     |42%             |50%             |0                    |2                    |20 NOV 2022|17 : 00|Group A |
|ENGLAND      |IRAN        |72%             |19%             |6                    |2                    |21 NOV 2022|14 : 00|Group B |
|SENEGAL      |NETHERLANDS |44%             |45%             |0                    |2                    |21 NOV 2022|17 : 00|Group A |
|UNITED STATES|WALES       |51%             |39%             |1                    |1                    |21 NOV 2022|20 : 00|Group B |
|ARGENTINA    |SAUDI ARABIA|64%             |24%

In [0]:

df2 = df2.withColumn("team1",lower(col("team1")))
df2 = df2.withColumn("team2",lower(col("team2")))
df2.select("team1", "team2").show(truncate=False)


+-------------+--------------+
|team1        |team2         |
+-------------+--------------+
|qatar        |ecuador       |
|england      |iran          |
|senegal      |netherlands   |
|united states|wales         |
|argentina    |saudi arabia  |
|denmark      |tunisia       |
|mexico       |poland        |
|france       |australia     |
|morocco      |croatia       |
|germany      |japan         |
|spain        |costa rica    |
|belgium      |canada        |
|switzerland  |cameroon      |
|uruguay      |korea republic|
|portugal     |ghana         |
|brazil       |serbia        |
|wales        |iran          |
|qatar        |senegal       |
|netherlands  |ecuador       |
|england      |united states |
+-------------+--------------+
only showing top 20 rows



In [0]:
df2 = df2.withColumn("team1", initcap(col('team1')))
df2 = df2.withColumn("team2", initcap(col('team2')))
df2.show(5, truncate=False)

+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+
|team1        |team2       |possession team1|possession team2|number of goals team1|number of goals team2|date       |time   |category|
+-------------+------------+----------------+----------------+---------------------+---------------------+-----------+-------+--------+
|Qatar        |Ecuador     |42%             |50%             |0                    |2                    |20 NOV 2022|17 : 00|Group A |
|England      |Iran        |72%             |19%             |6                    |2                    |21 NOV 2022|14 : 00|Group B |
|Senegal      |Netherlands |44%             |45%             |0                    |2                    |21 NOV 2022|17 : 00|Group A |
|United States|Wales       |51%             |39%             |1                    |1                    |21 NOV 2022|20 : 00|Group B |
|Argentina    |Saudi Arabia|64%             |24%

Postavljamo da nam kolona 'date' bude zaista datum, a ne string.

In [0]:
df2 = df2.withColumn("date", to_date(df2.date, 'dd MMM yyyy'))
df2.show(5, truncate=False)

+-------------+------------+----------------+----------------+---------------------+---------------------+----------+-------+--------+
|team1        |team2       |possession team1|possession team2|number of goals team1|number of goals team2|date      |time   |category|
+-------------+------------+----------------+----------------+---------------------+---------------------+----------+-------+--------+
|Qatar        |Ecuador     |42%             |50%             |0                    |2                    |2022-11-20|17 : 00|Group A |
|England      |Iran        |72%             |19%             |6                    |2                    |2022-11-21|14 : 00|Group B |
|Senegal      |Netherlands |44%             |45%             |0                    |2                    |2022-11-21|17 : 00|Group A |
|United States|Wales       |51%             |39%             |1                    |1                    |2022-11-21|20 : 00|Group B |
|Argentina    |Saudi Arabia|64%             |24%       

In [0]:
df2 = df2.withColumn("total goals",col("number of goals team1") + col("number of goals team2")) 
df2.show(5, truncate=False)    

+-------------+------------+----------------+----------------+---------------------+---------------------+----------+-------+--------+-----------+
|team1        |team2       |possession team1|possession team2|number of goals team1|number of goals team2|date      |time   |category|total goals|
+-------------+------------+----------------+----------------+---------------------+---------------------+----------+-------+--------+-----------+
|Qatar        |Ecuador     |42%             |50%             |0                    |2                    |2022-11-20|17 : 00|Group A |2          |
|England      |Iran        |72%             |19%             |6                    |2                    |2022-11-21|14 : 00|Group B |8          |
|Senegal      |Netherlands |44%             |45%             |0                    |2                    |2022-11-21|17 : 00|Group A |2          |
|United States|Wales       |51%             |39%             |1                    |1                    |2022-11-21|2

In [0]:
df2.printSchema()

root
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- possession team1: string (nullable = true)
 |-- possession team2: string (nullable = true)
 |-- number of goals team1: integer (nullable = true)
 |-- number of goals team2: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- category: string (nullable = true)
 |-- total goals: integer (nullable = true)



Spajamo kolone 'number of goals team1' i 'number of goals team2' u zajednicku 'final result' radi boljeg pregleda. Nakon toga dve pocetne kolone izabujemo, a mergeovanoj menjamo mesto u tabeli.

In [0]:
df2 = df2.withColumn("final result",concat(col("number of goals team1"),lit('-'),col("number of goals team2"))) 
df2.show(5, truncate=False)    

+-------------+------------+----------------+----------------+---------------------+---------------------+----------+-------+--------+-----------+------------+
|team1        |team2       |possession team1|possession team2|number of goals team1|number of goals team2|date      |time   |category|total goals|final result|
+-------------+------------+----------------+----------------+---------------------+---------------------+----------+-------+--------+-----------+------------+
|Qatar        |Ecuador     |42%             |50%             |0                    |2                    |2022-11-20|17 : 00|Group A |2          |0-2         |
|England      |Iran        |72%             |19%             |6                    |2                    |2022-11-21|14 : 00|Group B |8          |6-2         |
|Senegal      |Netherlands |44%             |45%             |0                    |2                    |2022-11-21|17 : 00|Group A |2          |0-2         |
|United States|Wales       |51%         

In [0]:
df2 = df2.drop("number of goals team1", "number of goals team2")
df2.show(5, truncate=False)    

+-------------+------------+----------------+----------------+----------+-------+--------+-----------+------------+
|team1        |team2       |possession team1|possession team2|date      |time   |category|total goals|final result|
+-------------+------------+----------------+----------------+----------+-------+--------+-----------+------------+
|Qatar        |Ecuador     |42%             |50%             |2022-11-20|17 : 00|Group A |2          |0-2         |
|England      |Iran        |72%             |19%             |2022-11-21|14 : 00|Group B |8          |6-2         |
|Senegal      |Netherlands |44%             |45%             |2022-11-21|17 : 00|Group A |2          |0-2         |
|United States|Wales       |51%             |39%             |2022-11-21|20 : 00|Group B |2          |1-1         |
|Argentina    |Saudi Arabia|64%             |24%             |2022-11-22|11 : 00|Group C |3          |1-2         |
+-------------+------------+----------------+----------------+----------

In [0]:
df2 =df2.select("team1","team2","final result", "total goals","possession team1","possession team2","date", "time", "category")
df2.show(10, truncate=False)    

+-------------+------------+------------+-----------+----------------+----------------+----------+-------+--------+
|team1        |team2       |final result|total goals|possession team1|possession team2|date      |time   |category|
+-------------+------------+------------+-----------+----------------+----------------+----------+-------+--------+
|Qatar        |Ecuador     |0-2         |2          |42%             |50%             |2022-11-20|17 : 00|Group A |
|England      |Iran        |6-2         |8          |72%             |19%             |2022-11-21|14 : 00|Group B |
|Senegal      |Netherlands |0-2         |2          |44%             |45%             |2022-11-21|17 : 00|Group A |
|United States|Wales       |1-1         |2          |51%             |39%             |2022-11-21|20 : 00|Group B |
|Argentina    |Saudi Arabia|1-2         |3          |64%             |24%             |2022-11-22|11 : 00|Group C |
|Denmark      |Tunisia     |0-0         |0          |55%             |33

Upisujemo podatke

In [0]:
path = 'dbfs:/projekat/'
csv_name = path + 'dfPlayers1.csv'
parquet_name = path + 'dfPlayers1.parquet'
orc_name = path + 'dfPlayers1.orc'

print(path,csv_name,parquet_name,orc_name,sep='\n')


df.write.csv(csv_name, header=True)
df_csv = spark.read.csv(csv_name, header=True)
df_csv.printSchema()
df_csv = spark.read.csv(csv_name, header=True, inferSchema=True)
df_csv.printSchema()


df.write.save(parquet_name, format='parquet') 
spark.read.load(parquet_name, format='parquet')


dbfs:/projekat/
dbfs:/projekat/dfPlayers1.csv
dbfs:/projekat/dfPlayers1.parquet
dbfs:/projekat/dfPlayers1.orc
root
 |-- player: string (nullable = true)
 |-- position: string (nullable = true)
 |-- national team: string (nullable = true)
 |-- age: string (nullable = true)
 |-- club: string (nullable = true)
 |-- birth_year: string (nullable = true)
 |-- games: string (nullable = true)
 |-- minutes: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- cards_yellow: string (nullable = true)
 |-- cards_red: string (nullable = true)
 |-- goals_per_game: string (nullable = true)

root
 |-- player: string (nullable = true)
 |-- position: string (nullable = true)
 |-- national team: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- club: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- games: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- a

Na kraju ucitavamo podatke o posecenosti utakmica. Menjamo velicinu slova imena kolona, neke kolone sredjujemo i na kraju pravimo novi raspored u tabeli.

In [0]:

file_location = "/FileStore/tables/FIFA_WC_2022_Attendance.csv"

df3 = spark.read.option("inferSchema", "true").option("header", "true").csv(file_location, sep=',')
df3.show(5, truncate=False)

+----------------+-------------+---------+------------+----------+-----------------------------+
|Date            |Time         |Home     |Away        |Attendance|Venue                        |
+----------------+-------------+---------+------------+----------+-----------------------------+
|20 November 2022|19:00 (21:30)|Qatar    |Ecuador     |67,372    |Al Bayt Stadium              |
|21 November 2022|16:00 (18:30)|England  |Iran        |45,334    |Khalifa International Stadium|
|21 November 2022|19:00 (21:30)|Senegal  |Netherlands |41,721    |Al Thumama Stadium           |
|21 November 2022|22:00 (00:30)|USA      |Wales       |43,418    |Ahmed bin Ali Stadium        |
|22 November 2022|13:00 (15:30)|Argentina|Saudi Arabia|88,012    |Lusail Iconic Stadium        |
+----------------+-------------+---------+------------+----------+-----------------------------+
only showing top 5 rows



In [0]:
from pyspark.sql import functions as F
df3 = df3.select([F.col(x).alias(x.lower()) for x in df3.columns])
df3.show(5, truncate=False)

+----------------+-------------+---------+------------+----------+-----------------------------+
|date            |time         |home     |away        |attendance|venue                        |
+----------------+-------------+---------+------------+----------+-----------------------------+
|20 November 2022|19:00 (21:30)|Qatar    |Ecuador     |67,372    |Al Bayt Stadium              |
|21 November 2022|16:00 (18:30)|England  |Iran        |45,334    |Khalifa International Stadium|
|21 November 2022|19:00 (21:30)|Senegal  |Netherlands |41,721    |Al Thumama Stadium           |
|21 November 2022|22:00 (00:30)|USA      |Wales       |43,418    |Ahmed bin Ali Stadium        |
|22 November 2022|13:00 (15:30)|Argentina|Saudi Arabia|88,012    |Lusail Iconic Stadium        |
+----------------+-------------+---------+------------+----------+-----------------------------+
only showing top 5 rows



In [0]:
df3 = df3.withColumn('time', split(df3['time'], ' ').getItem(0)) 
df3.show(5, truncate=False)

+----------------+-----+---------+------------+----------+-----------------------------+
|date            |time |home     |away        |attendance|venue                        |
+----------------+-----+---------+------------+----------+-----------------------------+
|20 November 2022|19:00|Qatar    |Ecuador     |67,372    |Al Bayt Stadium              |
|21 November 2022|16:00|England  |Iran        |45,334    |Khalifa International Stadium|
|21 November 2022|19:00|Senegal  |Netherlands |41,721    |Al Thumama Stadium           |
|21 November 2022|22:00|USA      |Wales       |43,418    |Ahmed bin Ali Stadium        |
|22 November 2022|13:00|Argentina|Saudi Arabia|88,012    |Lusail Iconic Stadium        |
+----------------+-----+---------+------------+----------+-----------------------------+
only showing top 5 rows



In [0]:
df3 =df3.select("home","away","attendance","venue","date","time")
df3.show(truncate=False) 

+-----------+--------------+----------+-----------------------------+----------------+-----+
|home       |away          |attendance|venue                        |date            |time |
+-----------+--------------+----------+-----------------------------+----------------+-----+
|Qatar      |Ecuador       |67,372    |Al Bayt Stadium              |20 November 2022|19:00|
|England    |Iran          |45,334    |Khalifa International Stadium|21 November 2022|16:00|
|Senegal    |Netherlands   |41,721    |Al Thumama Stadium           |21 November 2022|19:00|
|USA        |Wales         |43,418    |Ahmed bin Ali Stadium        |21 November 2022|22:00|
|Argentina  |Saudi Arabia  |88,012    |Lusail Iconic Stadium        |22 November 2022|13:00|
|Denmark    |Tunisia       |42,925    |Education City Stadium       |22 November 2022|16:00|
|Mexico     |Poland        |39,369    |Stadium 974                  |22 November 2022|19:00|
|France     |Australia     |40,875    |Al Janoub Stadium            |2

In [0]:
path = 'dbfs:/projekat/'
csv_name = path + 'dfAttendance1.csv'
parquet_name = path + 'dfAttendance1.parquet'
orc_name = path + 'dfAttendance1.orc'

print(path,csv_name,parquet_name,orc_name,sep='\n')


df3.write.csv(csv_name, header=True)
df3_csv = spark.read.csv(csv_name, header=True)
df3_csv.printSchema()
df3_csv = spark.read.csv(csv_name, header=True, inferSchema=True)
df3_csv.printSchema()


df3.write.save(parquet_name, format='parquet') 
spark.read.load(parquet_name, format='parquet')


dbfs:/projekat/
dbfs:/projekat/dfAttendance1.csv
dbfs:/projekat/dfAttendance1.parquet
dbfs:/projekat/dfAttendance1.orc
root
 |-- home: string (nullable = true)
 |-- away: string (nullable = true)
 |-- attendance: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)

root
 |-- home: string (nullable = true)
 |-- away: string (nullable = true)
 |-- attendance: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: timestamp (nullable = true)

Out[307]: DataFrame[home: string, away: string, attendance: string, venue: string, date: string, time: string]